In [1]:
# import records
# import sqlite3
# from urllib.parse import urljoin, urldefrag
import requests
from bs4 import BeautifulSoup
import re
from joblib import Parallel, delayed
import pickle
import csv
import MeCab as mc
import numpy as np

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree as tr

# import matplotlib.pyplot as plt
import random
from gensim.models import word2vec, LdaModel
from gensim import corpora
import gensim


In [ ]:
restaurant_list = []
for page in range(1,51):
    url = f'https://tabelog.com/rstLst/steak/{page}/?Srt=D&SrtT=rt&sort_mode=1&sk=%E3%82%B9%E3%83%86%E3%83%BC%E3%82%AD&svt=1900&svps=2&select_sort_flg=1'
    html_soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    for item in html_soup.find_all():
        
    restaurant_list.append(BeautifulSoup(requests.get(url).text, 'html.parser').find_all('a', class_='list-rst__rst-name-target cpy-rst-name'))

In [5]:
links = []
for restaurants in restaurant_list:
    for restaurant in restaurants:
        links.append(restaurant.get('href'))

In [1]:
tail = 'dtlrvwlst/?smp=0&lc=2&PG='
rev_results = []

# func = lambda review: [i.text for i in BeautifulSoup(requests.get('https://tabelog.com' + review).text, 'html.parser').find_all('div',class_='rvw-item__rvw-comment')]
def get_review(review):
    try:
        res = requests.get('https://tabelog.com' + review, timeout=5)
        return [i.text for i in BeautifulSoup(res.text, 'html.parser').find_all('div',class_='rvw-item__rvw-comment')]
    except Exception as e:
        print(e)


for link in links:
    if n_links % 50 == 0:
        np.savetxt("./texts/addition_700.csv", rev_results, fmt="%s", delimiter=",")
    iteration = 1
    while(True):
        url = link + tail + str(iteration)
        try:
            r = requests.get(url,timeout=5)
            if r.status_code != 200:
                break
            iteration += 1
            review_soup = BeautifulSoup(r.text, 'html.parser')
            review_urls =  [item.get('href') for item in review_soup.find_all('a', class_='rvw-simple-item__title-target')]
            rev_results.append(
                Parallel(n_jobs=4)([
                    delayed(get_review)(i) for i in review_urls
                ])
            )
        except Exception as e:
            print(e)
    

In [9]:
# レストラン名と評価のDict作成
# restaurant_names = [item.string for item in html_soup.find_all('a', class_='list-rst__rst-name-target cpy-rst-name')]
# ratings = [item.string for item in html_soup.find_all('span', class_='c-rating__val c-rating__val--strong list-rst__rating-val')]
# for k,v in zip(restaurant_names, ratings):
#     print(k,v)

In [2]:
# f = open('./texts/', 'wb')
# pickle.dump(rev_results, f)
# f.close()

# ロード時コマンド
# results = pickle.load(open('./texts/sample_steak_corpus.csv', 'rb'))

csv_file = open("./texts/restaurant_reviews.csv", "r", errors="", newline="" )

#リスト形式
f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)

results= [row for row in f]

In [3]:
csv_file = open("./texts/addition_700.csv", "r", errors="", newline="" )
#リスト形式
f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)

results2= [row for row in f]

In [4]:
rev_results = results + results2[1:]

In [172]:
def tokenizer(texts):
    t = mc.Tagger("-Ochasen")
    # 辞書ロードうまくいかない
    #t = mc.Tagger( '-Owakati -d /Users/tominagaryota/mecab-ipadic-neologd')
    t.parse('')
    output = []
    node =  t.parseToNode(texts)  
    while node:
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            output.append(node.surface)
        node = node.next
        if node is None:
            break
    return output

def mecab_analysis(texts):
    t = mc.Tagger("-Ochasen")
    # 辞書ロードうまくいかない
    #t = mc.Tagger( '-Owakati -d /Users/tominagaryota/mecab-ipadic-neologd')
    t.parse('')
    output = []
    node =  t.parseToNode(texts)  
    while node:
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            if word_type in ['名詞', '形容詞']:
                output.append(node.surface)
            #output.append(node.surface)
        node = node.next
        if node is None:
            break
    return output

In [ ]:
# sklearnでのLDA。使わないかも

# import mglearn
# from sklearn.decomposition import LatentDirichletAllocation

# vect = CountVectorizer(max_features=10000, max_df=.15)
# # mecab_analysisを使用
# X = vect.fit_transform(mecab_analysis([r[0] for r in rev_results[0]]))
# lda = LatentDirichletAllocation(n_topics=5, learning_method="batch", max_iter=25, random_state=0)
# document_topics = lda.fit_transform(X)

# sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# feature_names = np.array(vect.get_feature_names())
# mglearn.tools.print_topics(topics=range(5), feature_names=feature_names, 
#                            sorting=sorting, topics_per_chunk=1, n_words=5)

In [ ]:
# vect = CountVectorizer(max_features=10000, max_df=.15)
# N_TOPICS = 6
# # mecab_analysisを使用
# X = vect.fit_transform(mecab_analysis([r[0] for r in rev_results[1]]))
# lda = LatentDirichletAllocation(n_topics=N_TOPICS, learning_method="batch", max_iter=25, random_state=0)
# document_topics = lda.fit_transform(X)

# sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# feature_names = np.array(vect.get_feature_names())
# mglearn.tools.print_topics(topics=range(N_TOPICS), feature_names=feature_names, 
#                            sorting=sorting, topics_per_chunk=1, n_words=10)

In [8]:
all_reviews = []
for rvws in rev_results:
    for r in rvws:
        all_reviews.append(r)

In [9]:
len(all_reviews)

117619

In [84]:
csv_file = open("./texts/wakati.csv", "r", errors="", newline="" )

#リスト形式
f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)

wakati_text = [row for row in f]

In [85]:
new_wakati = []
for text in wakati_text:
    striped =  [t.strip("['n'").strip("'n']").strip("'") for t in text]
    new_wakati.append(striped)
wakati_text = new_wakati

In [2]:
# model = word2vec.Word2Vec(new_wakati, size=1000,min_count=5,window=5,iter=100,workers=4,sg=1)

In [ ]:
model.most_similar('ステーキ')

In [255]:
# モデルのセーブ
model.save("./model/sample_steak_model.model")

In [249]:
#np.savetxt("./texts/sample_steak_corpus.csv", wakati_text, fmt="%s", delimiter=",")

In [259]:
model.most_similar(positive=[u"挽肉",u"ハンバーグ"])

/Users/tominagaryota/.anyenv/envs/pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('エビフライ', 0.6223527789115906),
 ('肉汁', 0.5919903516769409),
 ('デミグラスソース', 0.5824424624443054),
 ('肉感', 0.5666879415512085),
 ('フライ', 0.5600730180740356),
 ('俵', 0.5347720384597778),
 ('パクリ', 0.5159295797348022),
 ('デミ', 0.5084888339042664),
 ('ジュワー', 0.5022367238998413),
 ('箸', 0.49991917610168457)]

In [88]:
dictionary = corpora.Dictionary(wakati_text)
dictionary.save_as_text('./texts/text.dict')

In [89]:
corpus = [dictionary.doc2bow(doc) for doc in wakati_text]
corpora.MmCorpus.serialize('./texts/text.mm', corpus)

In [90]:
tfidf = gensim.models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [91]:
with open('./model/corpus_tfidf.dump', mode='wb') as f:
    pickle.dump(corpus_tfidf, f)

In [93]:
tfidf_lda = gensim.models.LdaMulticore(corpus=corpus_tfidf, id2word=dictionary,
                             num_topics=10, minimum_probability=0.001,
                             passes=20, eval_every=0, chunksize=1000,workers=4)

In [94]:
tfidf_lda.save('./model/tfidf_lda.model')

In [24]:
# with open('model/tfidf_lda.model', mode='rb') as f:
#     tfidf_lda = pickle.load(f)
tfidf_lda = gensim.models.LdaModel.load('./model/tfidf_lda.model')

In [95]:
for i in range(10):
    print('tpc_{0}: {1}'.format(i, tfidf_lda.print_topic(i)[0:80]+'...'))

tpc_0: 0.006*"希少" + 0.004*"おにぎり" + 0.004*"松阪" + 0.003*"テキ" + 0.003*"浜松" + 0.003*"表参道" +...
tpc_1: 0.009*"u3" + 0.008*"000" + 0.003*"お肉" + 0.003*"の" + 0.003*"０" + 0.003*"店" + 0.00...
tpc_2: 0.011*"000" + 0.011*"1" + 0.007*"500円" + 0.007*"u3" + 0.007*"円" + 0.006*"2" + 0....
tpc_3: 0.016*"" + 0.006*"うし" + 0.004*"バンビーナ" + 0.003*"フライデーズ" + 0.003*"極み" + 0.003*"割烹"...
tpc_4: 0.010*"ハンバーグ" + 0.006*"ステーキ" + 0.004*"０" + 0.004*"肉" + 0.004*"ソース" + 0.004*"ライス"...
tpc_5: 0.012*"xa" + 0.010*"トンテキ" + 0.009*"0" + 0.007*"050" + 0.006*"スペアリブ" + 0.003*"108...
tpc_6: 0.012*"牛タン" + 0.004*"焼肉店" + 0.003*"西麻布" + 0.003*"卵黄" + 0.002*"網焼き" + 0.002*"センス"...
tpc_7: 0.005*"280円" + 0.005*"ミスジ" + 0.003*"トニーローマ" + 0.003*"ビアホール" + 0.003*"白子" + 0.003...
tpc_8: 0.006*"げんこつハンバーグ" + 0.005*"ラム" + 0.004*"海鮮" + 0.003*"新橋" + 0.003*"開放" + 0.003*"ハ...
tpc_9: 0.003*"レバ刺し" + 0.003*"ヘレ" + 0.002*"フランス料理" + 0.002*"ローストチキン" + 0.002*"伊勢" + 0.00...


In [352]:
# TF-IDF処理をしないLDA
# raw_lda = gensim.models.LdaMulticore(corpus=corpus, id2word=dictionary,
#                              num_topics=10, minimum_probability=0.001,
#                              passes=20, eval_every=0, chunksize=10000,workers=4)

In [44]:
temp ='天下茶屋でランチの機会があればその都度狙っていたこちらの超人気洋食店。売りは一頭買いするらしい黒毛を使ったお肉料理という触れ込みですわ。中でも黒バーグいうハンバーグがメインらしい情報は温めてました＾＾場所は天下茶屋駅から数分です。立ち飲み店がしのぎを削るサイドではない場所でテニススクールのあるあたり。健全でよろしおす(笑)。いうても、平日のランチなんで・・・とはいっても、一番安いハンバーグのランチでも1500円となかなかの高額です。それ相応のレベルは求められるはず。この日は1時前でしたが満席でした。かなり評判がいいみたいです。□黒バーグとカキフライ（1800円）○　エビフライなどの海鮮のフライも人気とのことで、カキフライを＾＾ｖ　まず、肝心の黒バーグですが、うん、僕は美味しいと思いました。　特徴的はなのは、スパイス含め、お肉に味がしっかりついてることと、　牛肉の香りも歯ごたえもあるけど、全然しつこくなくて、なおかつ、　デミグラスソースもあっさりしていて食べやすいです。脂こってりの　濃厚ソースみたいなハンバーグが好きな人には向かないけど、ただ、　　そうだとして、焼きも含めてくいしんぼー山中さんクラスのハンバーグが　といえばあまりに普通かなあ。デミグラスソースも2000円出すにしては。　カキフライは、大きさ重視です。タルタルはおいしいけど、揚げる牡蠣と　してはどうなんだろういう印象。このあたりが、叩き上げのシェフの洋食店　とは一線を画するといったところなのかな。　コンソメ・ご飯は普通にいけました＾＾確かに、この地域で見れば突出してるかもしれないですけど、作法・値段は大阪市内水準です。美味しいのは美味しいんですけどね＾＾ｖ昼に天下茶屋に行かれた際には是非。ご参考まで♪'

In [98]:
vec = dictionary.doc2bow(mecab_analysis(temp))
tfidf_lda[vec]

[(0, 0.016632209),
 (1, 0.46797404),
 (4, 0.48619828),
 (6, 0.0087923305),
 (8, 0.016394185)]

In [31]:
# LDA分類したベクトルをnumpyベクトル化
def convert_to_np_vec(vec):
    n_v = np.zeros(50)
    for v in vec:
        n_v[v[0]] = v[1]
    return n_v

In [32]:
# コサイン類似度を計算
def cos_sim(v1,v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [99]:
vec = dictionary.doc2bow(mecab_analysis(rvws[10]))
vec2 = dictionary.doc2bow(mecab_analysis(rvws[14]))
vec1 = convert_to_np_vec(tfidf_lda[vec])
vec2 = convert_to_np_vec(tfidf_lda[vec2])

In [101]:
cos_sim(vec1,vec2)

0.7385122087001152

In [3]:
data = word2vec.Text8Corpus("./wiki/wiki_wakati.txt")
model =  word2vec.Word2Vec(data, size=100)

In [104]:
model.most_similar(positive = ['純粋','悪'],negative = ['正義'])

/Users/tominagaryota/.anyenv/envs/pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('単純', 0.7389036417007446),
 ('低級', 0.7317520380020142),
 ('邪悪', 0.6965624690055847),
 ('不完全', 0.6847642660140991),
 ('無機質', 0.6803640127182007),
 ('鋭敏', 0.6790430545806885),
 ('複雑', 0.6772918701171875),
 ('微細', 0.676936686038971),
 ('獰猛', 0.6749590635299683),
 ('醜悪', 0.6736862063407898)]

In [111]:
model.most_similar(positive = ['繋がり'])

/Users/tominagaryota/.anyenv/envs/pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('つながり', 0.9462962746620178),
 ('関わり', 0.8196489810943604),
 ('結びつき', 0.8081238269805908),
 ('結び付き', 0.7764103412628174),
 ('関係性', 0.7676485776901245),
 ('かかわり', 0.7665578126907349),
 ('信頼関係', 0.7625255584716797),
 ('関連性', 0.7572896480560303),
 ('関係', 0.7516380548477173),
 ('係わり', 0.7378243207931519)]

In [112]:
model.most_similar(positive = ['志'])

/Users/tominagaryota/.anyenv/envs/pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('徳', 0.6933034658432007),
 ('義', 0.6846591234207153),
 ('吾', 0.681052029132843),
 ('慮', 0.6785447597503662),
 ('聚', 0.6748489737510681),
 ('鑑', 0.6738952994346619),
 ('祇', 0.6736379861831665),
 ('澄', 0.6709215641021729),
 ('為政', 0.6708956956863403),
 ('箕', 0.6702141761779785)]

In [115]:
model.most_similar(negative = ['感謝'])

/Users/tominagaryota/.anyenv/envs/pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('内蔵式ギアボックス', 0.42438018321990967),
 ('カスケードトンネル', 0.41108304262161255),
 ('R6RS', 0.40986916422843933),
 ('ドルトムント市電', 0.4021916091442108),
 ('バヨネットマウント', 0.3970351815223694),
 ('シンドラー社', 0.39160555601119995),
 ('スタジオ・ハーフ・アイ', 0.38931676745414734),
 ('メモリアロケータ', 0.3883298635482788),
 ('シビアコンディション', 0.3876619040966034),
 ('ASELSAN', 0.38623949885368347)]

In [116]:
import markovify

In [280]:
all_reviews = []
for rvws in rev_results:
    for r in rvws:
        all_reviews.append(r.strip("[['\\n\\n").strip(" ").strip("\n\\n']"))

In [298]:
tokenized = [mc.Tagger('-Owakati').parse(review) for review in all_reviews]

In [299]:
text = ''.join(tokenized)

In [290]:
def format_text(t):
    t = t.replace('　', ' ')  # Full width spaces
    # t = re.sub(r'([。．！？…]+)', r'\1\n', t)  # \n after ！？
    t = re.sub(r'(.+。) (.+。)', r'\1 \2\n', t)
    t = re.sub(r'\n +', '\n', t)  # Spaces
    t = re.sub(r'([。．！？…])\n」', r'\1」 \n', t)  # \n before 」
    t = re.sub(r'\n +', '\n', t)  # Spaces
    t = re.sub(r'\n+', r'\n', t).rstrip('\n')  # Empty lines
    t = re.sub(r'\n +', '\n', t)  # Spaces
    return t

In [302]:
text_model = markovify.NewlineText(format_text(text), 2)
for i in range(5):
    print(text_model.make_sentence())

東京 で 食べる と 生肉 の 塩 、 コショー 、 ナツメグ など の 柑橘系 の 爽やか な ハイボール で 乾杯 ！ 空腹 を 埋める 事 が 少ない 駅前通 に ある 、 知ら ない もっと いい のに 。 。 ＊ デザート ＊ カウンター から 調理 が 1人 で 食べ てる 方 も 、 800円 前後 の 方 に 連れ て 行く トコ じゃ なかっ た の だ 。 これ は ソース より 塩 っけ が 強く なり 、 移動 。 20時 くらい に 4 名 様 へ \ u3 000 ええ 肉 出す 店 が ある 。 値段 は とり ず らく 温厚 な 連れ が 、 サービス 別 ） 。 次に カット 場 で 味付け さ れ て いる だけ で クセ が 強い です 。 大きく 切っ て ださ れる 品々 でし た 。 見た目 や 色 々 頂き まし た ！ 色々 調べ た 結果 、 お つけもの から 提供 まで の 予約 を し て くれ た の だ が 、 ハンバーグ の 特徴 と 思わ れる 付近 の 席 と の 相性 良く て 、 セット の 方 が ウリ の 焼き アイスクリーム は 美味しく 感じ ます 。 全体的 に 油 が 飛び散る ので 紙 エプロン 用意 し て くれる が また れ て ます が 、 立地 と 駐車場 に 駐車 。 4 . 3 くらい しか 食べ ない こと に なる もの 今日 は どこ も 並ん でる 人達 が 多い 、 260g じゃ 物足りない か な と 。 味 変 として お 邪魔 し た 白米 に 吸わ せ た 方 が ほとんど い ない 。 ひたすら パン を 食べ て みる と 、 和牛 と オーストラリア 牛 が ええ っていう 訳 の わから ない の が 基本 です 。 女子会 や カップル など 客層 は やや 重かっ た 。 しかし 、 その 席 へ やっぱり 気 に なる 美味し さ で 驚き 。 これ が 実に よく このまま ご飯 に とても おすすめ です 。
うまい 生ビール を 注文 。 こちら の 方 から の そうだ ステーキ 行こう ！ 」 て 感じ た 。 うす 暗く て 写真 を 載せ た 「 そばめし 」 も おすすめ し ます 。 \ u3 000 思わず 「 今日

In [297]:
for _ in range(10):
    sentence = text_model.make_short_sentence(100, 20, tries=100).replace(' ', '')
    print(sentence)

食べる事ができました！美味しかったー！！ごちそうさまでした。
800円通常メニュー黒毛和牛100％フレッシュハンバーグ300ｇ2
以前からずーっと行きたいリストNO1だったミート矢澤さんに出掛けました。黒毛和牛100%デミグラスチーズハンバーグ2
500円黒毛和牛100%デミグラスチーズハンバーグ1
800円通常メニュー黒毛和牛100％フレッシュハンバーグ300ｇ2
150円黒毛和牛100%フレッシュハンバーグ\u3000桜が咲くころは\u3000Large2
u3000\u3000Regular3
五反田でお昼過ぎに商談が終わったのでハンバーグを食べさせてくれ。本当の評価はそれからだ。
800円通常メニュー黒毛和牛100％フレッシュハンバーグを注文です。黒毛和牛100%デミグラスチーズハンバーグ2
200円黒毛和牛100%フレッシュハンバーグを注文して¥4
